In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
from requests import Session
import requests

import pandas as pd
import time
import datetime
from tqdm import tqdm
import re
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

# db management libraries
import pymysql
from controller import MysqlController

In [2]:
# 연결
with open('../connection.txt', "r") as f:
                connect_info = list(map(lambda x: x.strip(), f.read().split(",")))
server = MysqlController(*connect_info)

# 주소지 받아오기
# ADR = pd.read_sql('SELECT DISTINCT CONCAT(sigungu, " ",dong) as adr FROM Address;', server.conn)
server.curs.execute('SELECT DISTINCT CONCAT(sigungu, " ",dong) as adr FROM Address;')
ADR = [f[0] for f in server.curs.fetchall()]

In [ ]:
server.conn.close()

# 식당 리스트 - Selenium

In [3]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Path
chromedriver_path = '/Users/yejin/Downloads/chromedriver'
driver = webdriver.Chrome(executable_path = chromedriver_path, options = options)

In [14]:
driver.quit()

In [4]:
# for adr in ADR.values
adr = ADR[0]
url = f'https://www.diningcode.com/list.php?query={adr}'
driver.get(url)

In [5]:
# 더보기? 10개 단위
res_list = [res for res in driver.find_elements_by_css_selector('#div_list > li') if type(res.get_property('onmouseenter')) == dict]

In [6]:
# for res in res_list:
res = res_list[0]
one_url = res.find_element_by_tag_name('a')

# 식당정보, 리뷰 - Selenium (하는김에)

In [7]:
# 클릭해서 열고 활성탭 옮김
one_url.click()
driver.switch_to.window(driver.window_handles[1])

In [12]:
driver.switch_to.window(driver.window_handles[1])

## DB Insert

In [13]:
# 식당정보
server.insert('diningcode_restaurants', line = {
    'updated_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'name': driver.find_element_by_css_selector('div.tit-point').text,
    'diningcode_id': re.findall('rid=(.*)', one_url.get_attribute('href'))[0], 
    'category': driver.find_element_by_css_selector('div.btxt').text.split('|')[1].strip(),
    'grade' : int(driver.find_element_by_css_selector('div.sns-grade strong').text.replace('점', "")),
    'star' : int(driver.find_element_by_css_selector('div.sns-grade span.point strong').text.replace('점', "")),
    'favorite' : int(driver.find_element_by_css_selector('div.favor-pic-appra i').text),
    'address' : driver.find_element_by_css_selector('li.locat').text,
    'phone' : driver.find_element_by_css_selector('li.tel').text
    # tag, char은 제외
})

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=96.0.4664.55)


In [ ]:
# 메뉴
menu = [m.text for m in driver.find_elements_by_css_selector('ul.list.Restaurant_MenuList li p.l-txt.Restaurant_MenuItem') if m.text != '']
# price = [int(re.sub('[원,]', "", p.text)) if re.sub('[원,]', "", p.text).isalnum() else p.text 
price = [p.text
            for p in driver.find_elements_by_css_selector('ul.list.Restaurant_MenuList li p.r-txt.Restaurant_MenuPrice') if p.text != '']

for i in menu:
    server.insert('diningcode_menu', line = {
        'updated_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'name': driver.find_element_by_css_selector('div.tit-point').text,
        'diningcode_id': re.findall('rid=(.*)', one_url.get_attribute('href'))[0],
        'menu' : menu[i],
        'price' : price[i]
    })

In [ ]:
# 안되면 NoSuchElementException
# 끝까지 더보기 누르고
while True:
    try:
        driver.find_element_by_css_selector('#div_more_review').click()
        time.sleep(1)
    except:
        break

In [ ]:
# 리뷰 크롤링
reviewers = [re.findall('(.*) [(](.*)[)]', pr.text)[0] for pr in driver.find_elements_by_css_selector('p.person-grade span.btxt')]
# point = [s.text for s in driver.find_elements_by_css_selector('p.point-detail')]
review = [r.text for r in driver.find_elements_by_css_selector('p.review_contents.btxt')]
date = [d.text for d in driver.find_elements_by_css_selector('span.star-date')]
star = [int(re.findall('[0-9]+', s)[0]) / 100 * 5 for s in driver.find_elements_by_css_selector('i.star > i').get_attribute('style')]

# if len(reviewers) != len(point): print("NOOOOOOO")

for i in range(len(reviewers)):
    # 리뷰
    server.inser('diningcode_reviews', line = {
        'updated_at': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'restaurant_name': driver.find_element_by_css_selector('div.tit-point').text,
        'diningcode_id': re.findall('rid=(.*)', one_url.get_attribute('href'))[0],
        'reviewer' : reviewers[i][0],
        'reviewer_info' : reviewers[i][1],
        'star' : star[i],
        # 'point_taste' : float(re.findall('맛([0-5][.]?[0-9]?)', point[i])[0]),
        # 'point_price' : float(re.findall('가격([0-5][.]?[0-9]?)', point[i])[0]),
        # 'point_service' : float(re.findall('서비스([0-5][.]?[0-9]?)', point[i])[0]),
        'review' : review[i],
        'date' : datetime.datetime.strptime(date[i], "%Y년 %m월 %d일")
    })

In [ ]:
# 창닫기
driver.close() 